## 1. ACCESSING DATA

In [4]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer



#pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


In [5]:
df = pd.read_csv('../Files/hr_raw_data_v1.csv')

In [6]:
df.head(1)


,employee_number,gender,birth_year,age,marital_status,dist_home,job_title,department,departured,year_at_comp,standard_hours,remote,business_travel,over_time,job_level,stock_opt_level,traning_times_last_year,perf_rate,year_last_promotion,year_current_mngr,education_field,education_scale,annual_salary,monthly_income,daily_rate,perc_salary_hike,env_sat_rate,job_involvement,job_sat_rate,relationship_sat_rate,work_life_balance,num_comp_worked,tot_working_year
0,1,0,1972,51,NaN,6,resEArch DIREcToR,NaN,No,20,Full Time,Yes,NaN,No,5,0,5,"3,0",15,15,NaN,3,"195370,00$","16280,83$",2015.722222,13,1,3,3,3,"3,0",7,NaN


## 2. DATA CLEANING

In [7]:
# Values format to title and strip

to_title = ['marital_status', 'job_title',  'department', 'education_field']

df[to_title] = df[to_title].apply(lambda x: x.str.title().str.strip() if x.dtype == 'object' else x)

In [8]:
# CONVERTING TO INTEGERS

to_integer = ['employee_number','age','dist_home','education_scale', 'job_involvement','job_level', 'job_sat_rate', 'num_comp_worked', 'perc_salary_hike', 'relationship_sat_rate', 'stock_opt_level', 'tot_working_year', 'work_life_balance','traning_times_last_year', 'year_at_comp', 'year_last_promotion', 'year_current_mngr']

def convert_int (data, list_columns):
    
        for col in list_columns:   
            try: 
                data[col] = data[col].str.replace(',','.')
                data[col] = pd.to_numeric(data[col], errors='coerce').astype('Int64')

            except:  data[col] = pd.to_numeric(data[col], errors='coerce').astype('Int64')
    

In [9]:
# Marital status typos fixing

df["marital_status"]=df["marital_status"].replace("Marreid", "Married")

In [10]:
# Converting obj to float

convert_float = ['annual_salary','monthly_income','daily_rate']
def replace_currency(value):
        try:
                value = float(value.replace("$", "").replace(",", "."))
                return value
        except:
            return value



for col in convert_float:
     df[col]=df[col].apply(replace_currency)

In [11]:
# Distance from home negatives to positives

df['dist_home'] = df['dist_home'].abs()

In [12]:
# Replacing values obj to numeric in age

age_update = {
    'thirty-two': 32,
    'twenty-four': 24,
    'thirty': 30,
    'fifty-eight': 58,
    'fifty-two': 52,
    'twenty-six': 26,
    'fifty-five': 55,
    'thirty-seven': 37,
    'thirty-six': 36,
    'forty-seven': 47,
    'thirty-one': 31}

df['age'] = df['age'].replace(age_update)

In [13]:
# Gender update, where 0 is male and 1 is female
gender_map = {0: "M", 1:"F"}
df['gender'] = df['gender'].map(gender_map)

df['gender'].unique()

array(['M', 'F'], dtype=object)

In [14]:
# Convereting and ensuring columns are integers

to_integer = ['employee_number','age','dist_home','education_scale', 'job_involvement','job_level', 'job_sat_rate', 'num_comp_worked', 'perc_salary_hike', 'relationship_sat_rate', 'stock_opt_level', 'tot_working_year', 'work_life_balance','traning_times_last_year', 'year_at_comp', 'year_last_promotion', 'year_current_mngr','perf_rate', 'tot_working_year']
def convert_int (data, list_columns):
        for col in list_columns:
            try:
                data[col] = data[col].str.replace(',','.')
                data[col] = pd.to_numeric(data[col], errors='coerce').astype('Int64')
            except:  data[col] = pd.to_numeric(data[col], errors='coerce').astype('Int64')
            
# Calling the convertion       
convert_int (df, to_integer)
        

## 3. NULLS MANAGEMENT

In [15]:
def impute_department(data_1, data_2):

#data_1 = x["department"] (data to be imputed) 
#dato_2 = x["job_title"] (data for reference on the imputation)
#dicc = {departamento1 : roles posibles dentro del departamento1, etc}

        dict ={"Research & Development": ["Healthcare Representative", "Laboratory Technician", "Manufacturing Director", "Research Scientist", "Research Director"],
                "Sales": ["Sales Executive", "Sales Representative"],
                "Human Resources": ["Human Resources"],
                "Unknown": ["Manager"]} #manager no tiene un departamento específico

        # Parte 1: Si dato_1 ya tiene un valor, lo mantenemos
        if isinstance(data_1, str):
                        return data_1

        # Parte 2: Si dato_1 es nulo, buscamos el jobrole en el diccionario:
        for k,v in dict.items():
            if data_2 in v: #completamos la celda de x['department'] con x["jobrole"]
                return k
        
        # Parte 3: Si no encontramos coincidencias, devolvemos "Unknown"
        return "Unknown"

In [16]:
# Apply the imputation function
df["department"] = df.apply(lambda x : impute_department(x["department"], x["job_title"]), axis=1)

In [17]:
# IMPUTATION WITH UNKNOWN

unknown_to_be_replaced = ['marital_status', 'business_travel', 'over_time', 'education_field']

def replace_unknown (data, list_columns):
    
    for col in list_columns:
        data[col] =  data[col].fillna('Unknown')
    

# EXECUTING THE FUNCCION:

replace_unknown(df, unknown_to_be_replaced)

In [18]:
# RELATED SALARY COLUMNS IMPUTATION

salary_base =['monthly_income', 'daily_rate', 'annual_salary']

imputer = KNNImputer(n_neighbors=3)
df[salary_base] = imputer.fit_transform(df[salary_base])

In [19]:
# AGE IMPUTATION

imputer = KNNImputer(n_neighbors = 5)
knn_imputed = imputer.fit_transform(df[['age', 'birth_year']])
df['age'] = knn_imputed

In [20]:
# CONFIRMATION: RELATED SALARY COLUMNS IMPUTATION 
df[salary_base].isnull().sum()

monthly_income    0
daily_rate        0
annual_salary     0
dtype: int64

In [21]:
df['remote'].unique()

array(['Yes', '1', 'False', '0', 'True'], dtype=object)

In [22]:
# REMOTE WORKER HOMOGENIZATION


remote_update = {'True':'Yes',
                 '1': 'Yes',
                'False': 'No',
                '0': 'No'}

df['remote'] = df['remote'].replace(remote_update )

In [23]:
departments_dict =  {"Research & Development": ["Healthcare Representative", "Laboratory Technician", "Manufacturing Director", "Research Scientist", "Research Director"],
                    "Sales": ["Sales Executive", "Sales Representative"],
                    "Human Resources": ["Human Resources"]}

df['department'] = df['department'].fillna(df['job_title'].map(departments_dict))



## EXPORTING NEW DATA

In [24]:
df.shape

(1614, 33)

In [27]:
df.to_csv("../files/hr_raw_data_v2.csv", index=False)
